# Approche Bayésienne

In [1]:
%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
wine = pd.read_csv('Datasets/whiteWine.csv', delimiter = ";")

In [3]:
d = 11  #descripteurs
N =  wine.shape[0] #produits à tester
R = 5 #experts

y = np.array(wine.ix[:,d+2:]) #labels des annotateurs
x = np.array(wine.ix[:,0:d]) #variables explicatives
w0 = np.random.rand(1,d)

## Initialisation

In [4]:
def init_mu(y):
	mu = []
	for i in range(0,N):
		mu.append(np.sum(y[i])/R)
	return mu

mu0 = init_mu(y)

def sigma(z):
    return 1/(1+np.exp(-z))

In [75]:
#Nos hypothèses
#Sensitivity
mean_prior1 = np.random.rand(R) #Quel pourcentage de bonnes réponses l'expert donne en moyenne ?
var_prior1 = np.random.rand(R) #Avec quelle incertitude ?
#Sensibility
mean_prior2 = np.random.rand(R) #Quel pourcentage de bonnes réponses l'expert donne en moyenne ?
var_prior2 = np.random.rand(R) #Avec quelle incertitude ?

#On en déduit les paramètres 
a_prior = np.random.rand(2,R) 
b_prior = np.random.rand(2,R)
for i in range(0,R):
    a_prior[0][i] = (-mean_prior1[i]**3 + mean_prior1[i]**2 -mean_prior1[i]*var_prior1[i]**2)/var_prior1[i]**2
    a_prior[1][i] = a_prior[0][i]*(1-mean_prior1[i])/mean_prior1[i]
    b_prior[0][i] = (-mean_prior2[i]**3 + mean_prior2[i]**2 -mean_prior2[i]*var_prior2[i]**2)/var_prior2[i]**2
    b_prior[1][i] = b_prior[0][i]*(1-mean_prior2[i])/mean_prior2[i]

gamma_prior = np.random.rand(d,d)
gamma_prior = np.linalg.inv(gamma_prior)

## Etape E

In [18]:
#E-step

def ai(alpha,y):
    a = []
    for i in range(0,N):
        proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        a.append(proda)
    return a

def bi(beta,y):
    b = []
    for i in range(0,N):
        prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
        b.append(prodb)
    return b

def pi(x,w):
    p = []
    for i in range(0,N):
        p.append(sigma(x[i].dot(w.T)))
    return p

def mui(a,b,p):
    mu = []
    for i in range(0,N):
        mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
    return mu

def E_step(x,y,alpha,beta,w):
    a = ai(alpha,y)
    b = bi(beta,y)
    p = pi(x,w)
    mu = mui(a,b,p)
    return mu


## Maximum Log - Likelihood Estimator

In [19]:
def logLikelihood(w, y, alpha, beta):
    a = ai(alpha, y)
    b = bi(beta, y)
    p = pi(x, w)
    
    #On calcule directement la log-vraissemblance.
    vraissemblance = 0
    for i in range(0,N):
        vraissemblance = vraissemblance + np.log((a[i]*p[i])+b[i]*(1-p[i]))
        
    return vraissemblance

## Étape M

In [59]:
#M-step

def alpha_function(mu,y,a_prior):
	alpha = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += mu[i]*y[i][j]
			tmp2 += mu[i]
		alpha.append((a_prior[0][j]-1+tmp1)/(a_prior[0][j]+a_prior[1][j]-2+tmp2))
	return alpha

def beta_function(mu,y,b_prior):
	beta = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += (1-mu[i])*(1-y[i][j])
			tmp2 += 1-mu[i]
		beta.append((b_prior[0][j]-1+tmp1)/(b_prior[0][j]+b_prior[1][j]-2+tmp2))
	return beta

def updateW(w,x, eta, mu, gamma_prior):
    g = 0
    for i in range(0,N):
        g += (mu[i] - sigma(x[i].dot(w.T)))*x[i]
    tmp = np.reshape(-gamma_prior.dot(w.T),g.shape) # (11,1) -> (11,)
    g += tmp
    
    H = np.zeros((d,d))
    for i in range(0,N):
        H -= sigma(x[i].dot(w.T))*(1-sigma(x[i].dot(w.T)))*((x[i].reshape(11,1))*(x[i].reshape(1,11)))
    H -= gamma_prior
    w = w - eta*np.linalg.inv(H).dot(g)
    return w

In [52]:
updateW(w0,x,0.1,mu0,gamma_prior)

(11,) (11,)


array([[ -9100.7203003 , -15164.56679015, -14641.21693821,  -8978.70023257,
        -21617.13761227, -32166.65899101, -22047.41529114, -21921.18188955,
        -11169.74619847,  -8542.44577971,  -7980.54860148]])

## Itérations

In [61]:
mu = init_mu(y)
alpha = alpha_function(mu,y,a_prior)
beta = beta_function(mu,y,b_prior)
diff_w = 10
w = w0

compteur = 0

#while (np.linalg.norm(diff_w) > 0.001) : # Limite de convergence à decider
while (compteur < 1000):
    mu = E_step(x,y,alpha,beta,w)
    alpha = alpha_function(mu,y,a_prior)
    beta = beta_function(mu,y,b_prior)
    w_bis = updateW(w,x,0.01,mu,gamma_prior)
    diff_w = w - w_bis
    w = w_bis
    if (compteur % 100 == 0):
        print ("ITERATION : ", compteur)
        print("Vraissemblance : ", logLikelihood(w, y, alpha, beta))
        print("Norme de diff_w : ", np.linalg.norm(diff_w))
        print("Alpha : ", alpha)
        print("Beta : ", beta)
    compteur = compteur + 1

ITERATION :  0
Vraissemblance :  [-12941.83783644]
Norme de diff_w :  0.0119657006494
Alpha :  [array([ 0.89407513]), array([ 0.70621429]), array([ 0.51095051]), array([ 0.40359964]), array([ 0.10282503])]
Beta :  [array([ 0.33729003]), array([ 0.52748231]), array([ 0.51300264]), array([ 0.99838001]), array([ 0.51638934])]
ITERATION :  100
Vraissemblance :  [-12941.84119002]
Norme de diff_w :  0.00334425755586
Alpha :  [array([ 0.89407512]), array([ 0.7062143]), array([ 0.51095052]), array([ 0.40359969]), array([ 0.10282503])]
Beta :  [array([ 0.33738594]), array([ 0.5274941]), array([ 0.51296967]), array([ 0.99823425]), array([ 0.51623876])]
ITERATION :  200
Vraissemblance :  [-12941.84371529]
Norme de diff_w :  0.00118212255319
Alpha :  [array([ 0.89407511]), array([ 0.70621431]), array([ 0.51095053]), array([ 0.40359969]), array([ 0.10282504])]
Beta :  [array([ 0.33741106]), array([ 0.52750347]), array([ 0.51293408]), array([ 0.99824062]), array([ 0.51617472])]
ITERATION :  300
Vrai

KeyboardInterrupt: 

In [ ]:
class model:
    """Classe qui encapsule l'apprentissage"""
    
    def __init__(self, maxIter = 10000, eta = 0.01):
        """Constructeur"""
        self.maxIter = maxIter
        self.eta = eta
        
    def initMu(self, y):
        """Initialisation de mu"""
        mu = []
        for i in range(0,N):
            mu.append(np.sum(y[i])/R)
        return mu
    
    def ai(self, alpha, y):
        """Update du vecteur a (1xN)"""
        a = []
        for i in range(0,N):
            proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
            a.append(proda)
        self.a = a
        
    def bi(self, beta, y):
        """Update du vecteur b (1xN)"""
        b = []
        for i in range(0,N):
            prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
            b.append(prodb)
        self.b = b
        
    def pi(self, x, w):
        """Update du vecteur p (1xN)"""
        p = []
        for i in range(0,N):
            p.append(sigma(x[i].dot(w.T)))
        self.p = p
        
    def mui(self, a,b,p):
        """Update de"""
        mu = []
        for i in range(0,N):
            mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
        self.mu = mu
    
    def EStep(x,y,alpha,beta,w):
        CE = 0 #Conditionnal excepectation
        a = ai(alpha,y)
        b = bi(beta,y)
        p = pi(x,w)
        mu = mui(a,b,p)
        for i in range(0,N):
            CE += mu[i]*np.log(p[i])*a[i]+(1-mu[i])*np.log(1-p[i])*b[i]
        

In [146]:
w0

array([[ 0.57957472,  0.71532897,  0.80352987,  0.62343161,  0.07593625,
         0.84190819,  0.5616814 ,  0.47498484,  0.13258545,  0.73825141,
         0.72939968]])